In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras

import time
from datetime import datetime, timedelta
from pylib import mnist_dataset
from pylib.tensorboardcmd import tensorboard_cmd

In [ ]:
#Note: increase this to at least 10 for a useful run, 20 or more produces better results
num_epochs = 10

<!-- requirement: images/VAE.png -->
<!-- requirement: pylib/__init__.py -->
<!-- requirement: pylib/tensorboardcmd.py -->
<!-- requirement: pylib/mnist_dataset.py -->
<!-- requirement: pylib/tf_utils.py -->

# Variational Autoencoders


## Autoencoders


Autoencoders are neural networks where the number of input and output neurons are the same. If our input neurons represent pixels in an image, the output of our autoencoder will ideally be the input image. Why would we want to create a model that simply reproduces our data? If we restrict the number of neurons in our hidden layers to be less than the number of input or output neurons, we force our model to learn sparse representations of the data. Therefore, autoencoders can be used for image compression and removing noise from images. 

![VAE](images/VAE.png)

An autoencoder consists of two neural networks -- an **encoder** and **decoder**. The encoder takes in high dimensional data and generates low dimensional representations of that data. Then, the decoder takes the low dimensional representations and translates them back into the high dimensional input space. 


## Variational Autoencoders (VAEs)


Variational Autoencoders (VAEs) differ from regular autoencoders, because they not only learn sparse representations of data but also generate new data. Consequently, VAEs are used to [create new images](https://openai.com/blog/generative-models/). For example, we can train a VAE on the MNIST data set and have it create an image of a handwritten "5." 

How do VAEs generate new data? They do so by making smart assumptions about the distributions of these sparse data representations, or **latent vectors**. More generally, they belong to a class of models called generative models, which learn the joint probability distribution between the input ($x$) and output (or latent vectors, $z$). We can then use this information to come up with likely $(x,z)$ pairs. For example, once we learn the distribution corresponding to the sparse representation of a handwritten "5," we can sample from this distribution to form new latent vectors for "5." 

Due to this constraint on the distributions of $z$, VAEs require an additional component in their loss function that penalizes deviations from these distributions.   

In this tutorial, we will build a simple VAE to recreate images in the MNIST data set. We will start off as usual by resetting our session, loading our data, setting our variables, and defining our weights and biases functions. 

In [ ]:
# Set the path to our summary logs
now = datetime.now()
logs_path = now.strftime("%Y%m%d-%H%M%S") + '/summaries'

In [ ]:
# Set parameters
img_size = 28
img_size_flat = img_size * img_size
img_shape = (img_size, img_size)

batch_size = 256
num_iterations =  2000
display_step = 100

In [ ]:
from pylib.tf_utils import mnist_test, mnist_train
train_images, train_label = mnist_train()
test_images, test_label = mnist_test()

## Encoder and decoder


As mentioned in above, VAEs make assumptions about the distribution of latent variables. Therefore, we will now consider VAEs from a probability framework. 

Recall **Bayes' Theorem** that tells us:

$$\begin{eqnarray}
P(z \mid x) &=& \frac{P(x \mid z) \cdot P(z)}{P(x)} \\
\\
\text{posterior distribution} &=& \frac{\text{likelihood} \times \text{prior}}{\text{marginal likelihood}}
\end{eqnarray}$$


Say we have data $x$ and latent variables $z$. The encoder tries to approximate the posterior distribution $P(z \mid x)$, or generate latent variables conditioned on the data. On the other hand, the decoder takes $z$ [sampled from $P(z \mid x)$] and outputs parameters to the likelihood distribution $P(x \mid z)$. These parameters are the weights and biases of the neural networks. 

Going back to the neural network framework, in the code below, our encoder and decoder are neural networks (of two layers each) that are mirror images of each other. We feed the output of the encoder directly into the decoder to make the full autoencoder.

We want to allow the flexibility to change the number of layers and their size without much work, so we'll build the layers up with a for loop through a list of sizes.

Note that we are using the functional interface for this.  This allows us to build two distinct models, the encoder and decoder, then build our full model by using the former as the input to the latter.  This way, when we train the big model, we train both the decoder and encoder without extra work.

In [ ]:
#We'll build a network that goes 784 -> 256 -> 128 -> 256 -> 784
layer_sizes = [256, 128]

original = keras.layers.Input(shape=(img_size_flat,))

#Build the forward part, 784 -> 256 -> 128
layer = original
for size in layer_sizes:
    layer = keras.layers.Dense(size, activation='sigmoid')(layer)
#This is our encoder  
encoder_out = layer

encoder = keras.models.Model(original, encoder_out)
#encoder.summary()

#The decoder will be the reverse, 128 -> 256 -> 784
#We'll need to reverse our layers, 
#drop the first one (it's input now), and add the final shape
encoded_input = keras.layers.Input(shape=layer_sizes[-1:])
layer = encoded_input
for size in layer_sizes[-2::-1] + [img_size_flat]:
    layer = keras.layers.Dense(size, activation='sigmoid')(layer)
decoder_out = layer

decoder = keras.models.Model(encoded_input, decoder_out)
#decoder.summary()
    
autoencoder_out = decoder(encoder(original))
autoencoder = keras.models.Model(original, autoencoder_out)

## KL-divergence


We want our network to be accurate, but we also want the latent variables to approximate the posterior distribution. The amount of information lost when approximating $P(z \mid x)$ is called the [Kullback-Leibler divergence](https://en.wikipedia.org/wiki/Kullback–Leibler_divergence), and we will use it to construct our loss function. To make our lives simple, we will choose the posterior distribution to be a unit normal and [calculate](http://allisons.org/ll/MML/KL/Normal/) the divergence accordingly. 

Why do we want our latent variables to approximate a certain distribution? We want them to be useful, so we impose this constraint. We can think of this as a form of  of regularization where we lose some fidelity to ensure we are capturing only important features. In other words, we want to build a model that can generate images and not just memorize them. A nice explanation of choosing latent variables can be found [here](http://kvfrans.com/variational-autoencoders-explained/).

We also want to minimize the loss due to inaccurate pixel values and must therefore create a component of the loss function that penalizes these errors. Our loss function would then be the sum of these two contributions.

We won't be doing that here, as it slows down training quite a bit - our two goals, accurately reproducing the pixels and shaping our latent variable distributions, are in tension.  In addition, this requires information from both the center layer (the latent vectors) and the final layer, making the encoding of the loss more involved.

## Adam optimizer


Unlike last time, we will use the Adam Optimizer instead of the Gradient Descent Optimizer. This optimizer uses a decaying learning rate. 

As a reminder, we can interpret the learning rate as the size of the step we take down a gradient of our loss function. If the step size is too large, we may never get to the minimum. A large learning rate will manifest itself as noise in our loss curve that never converges to a minimum point. However, if we have a very small step size, our model may take a long time to run. Ideally, we want to take large steps at the start of the training process and small steps towards the end. The [Adam Optimizer](https://arxiv.org/pdf/1412.6980v8.pdf) changes the learning rate for us. 

In [ ]:
autoencoder.compile(loss='mse', optimizer='adam')

Then we will train our model.  Again, we note that our input and output are the same - we're training a model that reproduces its input.

In [ ]:
autoencoder.fit(train_images, train_images, epochs=num_epochs, 
                batch_size=60, validation_data=(test_images,test_images))

Finally we will regenerate 10 of the MNIST images. We will display the original test images above the regenerated ones. 

In [ ]:
n_examples = 10

# Applying encode and decode over test set
encode_decode = autoencoder.predict(test_images[:n_examples])

# Compare original images with their reconstructions
f, a = plt.subplots(2, n_examples, figsize=(20, 4))
for i in range(n_examples):
    a[0][i].imshow(np.reshape(test_images[i], img_shape))
    a[1][i].imshow(np.reshape(encode_decode[i], img_shape))

## Application: Noise removal

Our system now has the capacity to reproduce the original images, and thus has some sort of internal model of what they should look like.  We can take advantage of this by feeding it images that _almost_ look like what it's seen before - it will return its best guess as to what they should be, given what it's seen.  Since we've trained it on clean images, we expect it will return clean images.  So, if we feed it a noisy image, it should do its best to return something like it's seen before, often very successfully removing the noise.

Here we'll take an image, add a uniform random number to a percentage of the pixels, then run it through the encode-decode pathway.  We'll also run the clean image through the pathway to compare

In [ ]:
import random
def uniform_noise(img, percent):
    start = test_images[img].copy()
    noisy = start.copy()
    for i in range(len(test_images[img])):
        if random.random() < percent:
            noisy[i] += random.random()
            noisy[i] = min(noisy[i],1)
    f, a = plt.subplots(1, 4, figsize=(20, 4))
    encode_decode = autoencoder.predict(np.array([start, noisy]))
    a[0].imshow(np.reshape(start, img_shape))
    a[0].set_title("Input")
    a[0].grid(False)
    a[1].imshow(np.reshape(encode_decode[0], img_shape))
    a[1].set_title("Encode-decode on input")
    a[1].grid(False)
    a[2].imshow(np.reshape(noisy, img_shape))
    a[2].set_title("Noisy")
    a[2].grid(False)
    a[3].imshow(np.reshape(encode_decode[1], img_shape))
    a[3].set_title("Encode-decode on noisy")
    a[3].grid(False)

Here we've made a little function to do just that, just give it which test image (there are $10000$ of them) and what percentage of the pixels to add noise to

In [ ]:
from ipywidgets import interact
interact(uniform_noise, img=(0,100), percent=(0,0.4,0.05));

## Generating new images

Ideally, we'd be able to take a random draw from a 128-dimensional Gaussian of unit variance, and this would give us a new image.  That's the "variational" aspect, and enforced by the KL-divergence.  This turns out not to work particularly well for the data here.

Instead, we'll cheat.  We don't know the underlying 128-dimensional distribution of our data, but we can see what happens when we manipulate the compressed representation.  Let's try making a new 4 by making a linear combination of two old 4's.  

We have created our model in two pieces, an encoder and a decoder, and stitched them together.  Now we'll use them separately.

In [ ]:
first4 = 4
second4 = 210
four1 = test_images[first4]
four2 = test_images[second4]
def plot_images(im1, im2):
    fig, ax = plt.subplots(1, 2)
    ax[0].imshow(im1.reshape(28,28))
    ax[1].imshow(im2.reshape(28,28))
plot_images(four1, four2)

Let's see how the model does reconstructing them before we move forward.

In [ ]:
encode_decode = autoencoder.predict(test_images[[first4, second4]])

plot_images(encode_decode[0], encode_decode[1])

Now, we get their encoded representations and linearly interpolate between them.

In [ ]:
encode_im = encoder.predict(test_images[[first4, second4]])
code1 = encode_im[0]
code2 = encode_im[1]

In [ ]:
n_examples = 5
new4 = decoder.predict(np.array([code1*(1-p) + code2*p for p in np.linspace(0,1,n_examples)]))
f, a = plt.subplots(1, n_examples, figsize=(20, 6))
for i in range(n_examples):
    a[i].imshow(np.reshape(new4[i], img_shape))

## Exercise: New numbers

Generate a few new numbers as combinations of the old ones (they don't have to be linear combinations of two of them, you could do any size set, but remember that each pixel must be between 0 and 1).

## Exercise: More compression

We have chosen a hidden size of 128, which is about a seventh of the original size, and it does very well.  Try playing with this size and see what happens if you make it smaller.  We tried with 10 and got surprisingly good results.  Why might this be?

*Copyright &copy; 2018 The Data Incubator.  All rights reserved.*